# SweetDeals

The first cell is where parameters can be configured for which trips to investigate, the number of weeks to look into the future, which day of the week the trip will begin, and how many days the trip will last.  Set the parameters that are desired and then run the entire workbook.

In [1]:
itinerary_list = []
itinerary_list = [
#     {
#         "airport_list": ["SNA","PHX"], # Enter the origination and destination airport codes as strings in a list
#         "depart_day": 3, # 3 indicates a departure on Thursday
#         "trip_length": 4, # This value indicates how many days after departure the return flight will occur
#         "number_of_weeks": 27 # The total number of weeks to look into the future
#     },
#     {
#         "airport_list": ["LAX","ORD"], # Enter the origination and destination airport codes as strings in a list
#         "depart_day": 3, # 3 indicates a departure on Thursday
#         "trip_length": 4, # This value indicates how many days after departure the return flight will occur
#         "number_of_weeks": 27 # The total number of weeks to look into the future
#     },
    {
        "airport_list": ["SFO","IAD"], # Enter the origiteamnation and destination airport codes as strings in a list
        "depart_day": 3, # 3 indicates a departure on Thursday
        "trip_length": 4, # This value indicates how many days after departure the return flight will occur
        "number_of_weeks": 27 # The total number of weeks to look into the future
    },
#     {
#         "airport_list": ["SJC","MSY"], # Enter the origination and destination airport codes as strings in a list
#         "depart_day": 3, # 3 indicates a departure on Thursday
#         "trip_length": 4, # This value indicates how many days after departure the return flight will occur
#         "number_of_weeks": 27 # The total number of weeks to look into the future
#     },
#     {
#         "airport_list": ["LAX","MUC"], # Enter the origination and destination airport codes as strings in a list
#         "depart_day": 4, # 4 indicates a departure on Friday
#         "trip_length": 4, # This value indicates how many days after departure the return flight will occur
#         "number_of_weeks": 27 # The total number of weeks to look into the future
#     },
#     {
#         "airport_list": ["BOS","SNA"],
#         "depart_day": 4, # 4 indicates a departure on Friday
#         "trip_length": 9, # This value indicates how many days after departure the return flight will occur
#         "number_of_weeks": 27 # The total number of weeks to look into the future
#     }
]

In [2]:
#Import Dependencies for scraping
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
from dotenv import load_dotenv
from selenium import webdriver
from splinter import Browser
import pandas as pd
import numpy as np
import regex as re
import datetime
import random
import time

# Import dependencies for data cleaning
from sqlalchemy import create_engine
import os

load_dotenv()

db_password = os.getenv("db_password")
db_username = os.getenv("db_username")
db_url = os.getenv("db_url")
db_port = os.getenv("db_port")
db_name = os.getenv("db_name")
db_table = os.getenv("db_table")

In [3]:
def button_press(action):
    if action == "keystroke":
        return random.uniform(.091315816564,.15978136531)
    elif action == "enter_search":
        # This value needs to stay above 30 seconds to accommodate for the pause the website imposes when performing a search
        return random.uniform(30.2551845131843, 33.3189651876)
    elif action == "fare_click":
        return random.uniform(8.9098741610684,10.3498135130)
    else:
        return random.uniform(.2098741610684,.3498135130)

In [4]:
def get_dates(depart_day,trip_length,number_of_weeks):
    
    #Get today's date and add one so that we don't search for same day flights
    today = datetime.date.today()+datetime.timedelta(1)
    
    i = 0
    depart_ = []
    return_ = []
    for x in range(0,number_of_weeks):
        # Get the next possible departure date based on the desired day of the week to leave
        depart_.append((today + datetime.timedelta((depart_day-today.weekday()) % 7 )).strftime('%m/%d/%Y'))
        return_.append((today + datetime.timedelta((depart_day-today.weekday()) % 7 + trip_length)).strftime('%m/%d/%Y'))
        today = today + datetime.timedelta((depart_day-today.weekday()) % 7 + 7)
        i += 1
#         print(f"{depart_[i]} : {return_[i]}")
        
    return depart_,return_

In [5]:
def move_and_click(element):
    action.move_to_element(element).perform()
    time.sleep(button_press(0))
    
    action.click(element).perform()
    time.sleep(button_press(0))   

In [6]:
def set_time_of_day(element):
    
    # Select the element
    move_and_click(element)
        
    # move up to "All Day"
    for x in range(0,7):
        action.send_keys(Keys.UP)
        time.sleep(button_press("keystroke"))

    action.click(element).perform()
    time.sleep(button_press(0))

In [7]:
def enter_airport(airport,element):

    move_and_click(element)

    # Delete any 3-letter airport codes that may be currently in the field
    for x in range(0,3):
        action.send_keys(Keys.BACKSPACE).perform()
        time.sleep(button_press("keystroke"))

    # write in the new airport code
    for letter in airport:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))

    time.sleep(button_press(0))  

In [8]:
def enter_dates(date,element):

    move_and_click(element)
    
    for x in range(0,10):
        action.send_keys(Keys.BACKSPACE).perform()
        time.sleep(button_press("keystroke"))
    
    for letter in date:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))
        
    time.sleep(button_press(0))  

In [9]:
def execute_search(itinerary_list):

    # navigate to the website
    driver.get("https://www.aa.com/booking/find-flights")
    time.sleep(button_press(0))
    
    for itinerary in itinerary_list:
        
        # Set Time of Day for Departure and Return
        set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelTime"]'))
        set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelTime"]'))
        
        # Set departure airport
        enter_airport(itinerary['airport_list'][0],driver.find_element(by=By.XPATH,value=('//*[@id="segments0.origin"]')))

        # Set the destination airport
        enter_airport(itinerary['airport_list'][1],driver.find_element(by=By.XPATH,value=('//*[@id="segments0.destination"]')))
        
        # Get the list of all possible depart/return dates for the specified number of weeks
        depart_dates,return_dates = get_dates(itinerary['depart_day'],itinerary['trip_length'],itinerary['number_of_weeks'])
    
        for (depart_date,return_date) in zip(depart_dates,return_dates):

            # Set the travel dates
            enter_dates(depart_date,driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelDate"]'))
            enter_dates(return_date,driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelDate"]'))

            # Hitting the ENTER key while still in the airports field will execute a search
            action.send_keys(Keys.ENTER).perform()
            time.sleep(button_press("enter_search"))

            # Create a list to be the new line of the dataframe
            new_df_line = [datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                           f"{itinerary['airport_list'][0]}-{itinerary['airport_list'][1]}-{depart_date}-{return_date}",
                           itinerary['airport_list'][0],
                           itinerary['airport_list'][1],
                           depart_date,
                           return_date
                          ]

            try:
                # Read the fares that have been generated by the search
                departing_fare_element,returning_fare_element = read_fares()

                # Build the rest of the line in the new dataframe
                build_df(new_df_line,departing_fare_element,returning_fare_element)
            except Exception as e:
                print(f"An error ({e}) has occurred with {new_df_line[1]}")

            # navigate to the website
            driver.get("https://www.aa.com/booking/find-flights")
            time.sleep(button_press(0))




In [10]:
def get_trip_length(x):
    x = re.findall("<li.*?>",str(x.prettify()))[0]
    x = re.findall("data-triptime=\".*?\"",x)[0]
    return float(re.findall("[0-9]+",x)[0])

In [11]:
def get_max_trip_length(list_elements):
    
    shortest_trip_length = 2**15 # minutes
    for x in list_elements:
        x = get_trip_length(x)
        if x < shortest_trip_length:
            shortest_trip_length = x
    
    # After finding the shortest trip, return a maximum trip length of 2x the shortest
    return shortest_trip_length * 2

In [12]:
def trip_duration_filter(list_elements):
    
    # Get the maximum trip length
    max_trip_length = get_max_trip_length(list_elements)
    
    i = 0
    duration_filtered_elements = []
    for x in list_elements:
        y = get_trip_length(x)
        if y < max_trip_length:
            duration_filtered_elements.append(x)
        i += 1
    
    return duration_filtered_elements

In [13]:
def get_lowest_fare(list_elements):
    i = 0
    lowest_fare = 2**15
    for x in list_elements:
        x = re.findall("<li.*?>",str(x.prettify()))[0]
        new_lowest_fare = re.findall("data-lowestfare=\".*?\"",x)[0]
        new_lowest_fare = float(re.findall("[0-9]+",new_lowest_fare)[0])
        if new_lowest_fare < lowest_fare:
            lowest_fare = new_lowest_fare
            lowest_fare_element = list_elements[i]
        i += 1
    return lowest_fare_element

In [14]:
def get_fare_button(departing_fare_element):
    
    lowest_fare_element = re.findall("<button.*?MainCabin.*?>",str(departing_fare_element.prettify()))[0]
    lowest_fare_id = re.findall("id=\"slice\dFlight[0-9]+MainCabin\"",lowest_fare_element)[0]
    lowest_fare_id = f"//*[@{lowest_fare_id}]"

    return driver.find_element(by=By.XPATH,value=lowest_fare_id)

In [15]:
def sort_by_price():
    # Select the sorting menu
    sort_menu_element = driver.find_element(By.XPATH,value='//*[@id="searchResultsSortOptions"]')
    move_and_click(sort_menu_element)

    # Move to "Price (low to high)". Scroll to top and then back down 2 lines
    for x in range(0,8):
        action.send_keys(Keys.UP)
        time.sleep(button_press("keystroke"))

    for x in range(0,2):
        action.send_keys(Keys.DOWN)
        time.sleep(button_press("keystroke"))

    action.click(sort_menu_element).perform()
    time.sleep(button_press(0))

In [16]:
def build_df(new_df_line,departing_fare_element,returning_fare_element):
    
    # Set the pattern for finding everything between quotation marks
    pattern = "\"(.*?)\""

    # Add the total price data from the return flight to the list
    new_df_line.append(re.findall(pattern,re.findall("data-tripprice=\".*?\"",str(returning_fare_element))[0])[0])

    # Add the departing flight data to the list
    new_df_line.append(re.findall(pattern,re.findall("data-departuretime=\".*?\"",str(departing_fare_element))[0])[0])
    new_df_line.append(re.findall(pattern,re.findall("data-triptime=\".*?\"",str(departing_fare_element))[0])[0])
    new_df_line.append(re.findall(pattern,re.findall("data-tripstops=\".*?\"",str(departing_fare_element))[0])[0])

    # Add the returning flight data to the list
    new_df_line.append(re.findall(pattern,re.findall("data-departuretime=\".*?\"",str(returning_fare_element))[0])[0])
    new_df_line.append(re.findall(pattern,re.findall("data-triptime=\".*?\"",str(returning_fare_element))[0])[0])
    new_df_line.append(re.findall(pattern,re.findall("data-tripstops=\".*?\"",str(returning_fare_element))[0])[0])
    new_df_line
    
    # Write the new dataframe line to the next line in the dataframe
    df.loc[len(df)] = new_df_line
#     df.head()

In [17]:
def read_fares():
    
    # Get the html from the page
    html_depart = driver.page_source
    html_soup_depart = soup(html_depart,'html.parser')
#     button_press("keystroke")
    
    # Identifying the list elements that contain flight and price information
    depart_elements = trip_duration_filter(html_soup_depart.find_all("li",id=re.compile("slice0Flight[0-9]+")))
    
    # Get the element for the lowest fare on the departure page
    departing_fare_element = get_lowest_fare(depart_elements)
    
    # Get the button for the lowest fare element
    departing_fare_button = get_fare_button(departing_fare_element)

    # sort the results by price, so that the lowest price is no longer hidden
    sort_by_price()
    
    # Click the button for the lowest fare flight option
    move_and_click(departing_fare_button)
    button_press("fare_click")
    
    driver.refresh()
    button_press("fare_click")
        
    # Get the html from the page
    html_return = driver.page_source
    html_soup_return = soup(html_return,'html.parser')
    
    # Identifying the list elements that contain flight and price information
    return_elements = trip_duration_filter(html_soup_return.find_all("li",id=re.compile("slice1Flight[0-9]+")))
    
    # Get the element for the lowest fare on the return page
    returning_fare_element = get_lowest_fare(return_elements)
    
    return departing_fare_element,returning_fare_element

In [18]:
def data_ETL(df):
    # Force all data types to align with what the db is expecting
    df["total_cost"] = df["total_cost"].astype(float).astype(int)
    df["depart_stops"] = df["depart_stops"].astype(float).astype(int)
    df["return_stops"] = df["return_stops"].astype(float).astype(int)
    df["depart_duration"] = df["depart_duration"].astype(float).astype(int)
    df["return_duration"] = df["return_duration"].astype(float).astype(int)

    df["time_stamp"] = pd.to_datetime(df["time_stamp"])
    df["depart_datetime"] = pd.to_datetime(df["depart_datetime"])
    df["return_datetime"] = pd.to_datetime(df["return_datetime"])
    
    # Calculate and add the number of days before purchase the data were analyzed
    df['days_before_purchase'] = (pd.to_datetime(df['depart_date']) - df['time_stamp']).dt.days.astype(int).apply(lambda x: 0 if (x < 0) else x)
    
    #create the connection to the PostgreSQL database, then add the movies_df DataFrame to a SQL database.
    db_string = f"postgresql://{db_username}:{db_password}@{db_url}:{db_port}/{db_name}"
    engine = create_engine(db_string)
    df.to_sql(name=db_table, con=engine, if_exists='append',index=False)
    
    # Save the DataFrame to a .csv file
    airport_codes = ""
    for itinerary in itinerary_list:
        airport_codes = airport_codes+"_"+itinerary['airport_list'][0]+itinerary['airport_list'][1]
    path = f"Data/{analysis_time}{airport_codes}.csv"
    df.to_csv(path,index=False)

In [19]:
# Keeping the dataframe as a global entity allows it to be updated within the read_fare function
columns = ["time_stamp","trip_id","depart_airport","return_airport","depart_date","return_date","total_cost","depart_datetime","depart_duration","depart_stops","return_datetime","return_duration","return_stops"]
# columns = ["trip_id"]
df = pd.DataFrame(columns=columns)

# create webdriver object
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# create action chain object
action = ActionChains(driver)

# Get the timestamp for the save file
analysis_time = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

# Execute the search with itineraries as a list of lists
execute_search(itinerary_list)


# Close the driver
driver.close()

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


An error (local variable 'lowest_fare_element' referenced before assignment) has occurred with SFO-IAD-11/10/2022-11/14/2022


In [20]:
# Save the DataFrame to the db and a .csv for backup purposes
data_ETL(df)

In [21]:
df.tail(10)

,time_stamp,trip_id,depart_airport,return_airport,depart_date,return_date,total_cost,depart_datetime,depart_duration,depart_stops,return_datetime,return_duration,return_stops,days_before_purchase
16,2022-07-10 13:17:18,SFO-IAD-11/03/2022-11/07/2022,SFO,IAD,11/03/2022,11/07/2022,498,2022-11-03 07:00:00,300,0,2022-11-07 16:05:00,360,0,115
17,2022-07-10 13:19:06,SFO-IAD-11/17/2022-11/21/2022,SFO,IAD,11/17/2022,11/21/2022,547,2022-11-17 11:30:00,543,1,2022-11-21 16:05:00,368,0,129
18,2022-07-10 13:20:05,SFO-IAD-11/24/2022-11/28/2022,SFO,IAD,11/24/2022,11/28/2022,667,2022-11-24 08:14:00,418,1,2022-11-28 16:05:00,368,0,136
19,2022-07-10 13:20:59,SFO-IAD-12/01/2022-12/05/2022,SFO,IAD,12/01/2022,12/05/2022,498,2022-12-01 07:00:00,301,0,2022-12-05 16:05:00,368,0,143
20,2022-07-10 13:21:53,SFO-IAD-12/08/2022-12/12/2022,SFO,IAD,12/08/2022,12/12/2022,467,2022-12-08 07:00:00,301,0,2022-12-12 19:06:00,495,1,150
21,2022-07-10 13:22:46,SFO-IAD-12/15/2022-12/19/2022,SFO,IAD,12/15/2022,12/19/2022,675,2022-12-15 11:30:00,543,1,2022-12-19 16:05:00,368,0,157
22,2022-07-10 13:23:45,SFO-IAD-12/22/2022-12/26/2022,SFO,IAD,12/22/2022,12/26/2022,836,2022-12-22 23:55:00,459,1,2022-12-26 16:05:00,368,0,164
23,2022-07-10 13:24:39,SFO-IAD-12/29/2022-01/02/2023,SFO,IAD,12/29/2022,01/02/2023,718,2022-12-29 07:00:00,301,0,2023-01-02 16:05:00,368,0,171
24,2022-07-10 13:25:34,SFO-IAD-01/05/2023-01/09/2023,SFO,IAD,01/05/2023,01/09/2023,465,2023-01-05 11:30:00,543,1,2023-01-09 16:05:00,368,0,178
25,2022-07-10 13:26:34,SFO-IAD-01/12/2023-01/16/2023,SFO,IAD,01/12/2023,01/16/2023,528,2023-01-12 07:00:00,301,0,2023-01-16 16:05:00,368,0,185
